In [1]:
# Import libraries

from    pulp    import *  
import  pandas  as pd        # Will be used to generate and export tables                        


In [2]:
# Problem

model = LpProblem(  name    = "aggregate-planning", 
                    sense   = LpMinimize            )
                    

In [3]:
# Sets

T       = range(1,13)                   
T_ext   = range(13)                     
M       = ("b", "p")                    


In [4]:
# Parameters

D = (   (28, 14), 
        (20, 10),
        (26, 4),
        (24, 4),
        (18, 6),
        (10, 8),
        (22, 10),
        (20, 12),
        (18, 6),
        (32, 8),
        (34, 14),
        (36, 6)     )           

wInit   = 86                    
h       = 800                   
f       = 1400                  

c       = (250, 500)            
iInit   = (8, 3)                

pCost   = (6250, 9750)          

pWf     = (380, 530)            
p       = 180                   

r       = 420                   
o       = (420/180)*1.5*40      

oLim    = 40                    


In [5]:
# New parameters

e       = 1             # Cost of renting extra parking space       [$ per sq meter]
gp      = 500           # Total capacity of company's goods park    [sq meters]   
sr      = (40, 60)      # Space requirements                        [sq meters]

In [6]:
# Decision variables

X = {   (i, t): LpVariable( name="X-{model}-{month}".format(    model   = M[i],                                     
                                                                month   = t     if t>=10 
                                                                                else "0{}".format(t)),              
                            lowBound=0                                                                 )
                                                                                for i in range(len(M)) for t in T       }


I = {   (i, t): LpVariable( name="I-{model}-{month}".format(    model   = M[i],                                     
                                                                month   =   t   if t>=10 
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for i in range(len(M)) for t in T_ext   }


W = {   (t): LpVariable(    name="W-{}"             .format(                t   if t>=10                            
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for t in T_ext                          }


H = {   (t): LpVariable(    name="H-{}"             .format(                t   if t>=10                            
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for t in T                              }


F = {   (t): LpVariable(    name="F-{}"             .format(                t   if t>=10                           
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for t in T                              }


O = {   (t): LpVariable(    name="O-{}"             .format(                t   if t>=10                            
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for t in T                              }



In [7]:
# New decision variable: amount of extra space rented

E = {   (t): LpVariable(    name="E-{}"             .format(                t   if t>=10                            
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for t in T                              }

In [8]:
# Modified objective function

model += lpSum(
                [   pCost[i]*X[(i,t)]   +                                    
                    c[i]*I[(i,t)]                                            
                                            for t in T                       
                                            for i in range(len(M)   )   ] +  
                [   r*W[t] + o*O[t]     +                                    
                    h*H[t] + f*F[t]     +
                    e*E[t]                                          
                                            for t in T                  ]    
                                                                            )

In [9]:
# Constraints


model += (W[0]      ==  wInit                                                                               
                                                                                    )


for i in range(len(M)):                                                                         
    model += (I[(i, 0)] ==  iInit[i]
                                                                                    )


for i in range(len(M)):                                                                         
    for t in T:
        model += (  I[(i,t-1)] + X[(i,t)] == I[(i,t)] + D[t-1][i]
                                                                                    )


for t in T:                                                                                     
    model += (  W[t] == W[t-1] + H[t] - F[t]
                                                                                    )


for t in T:                                                                                     
    model += (  O[t] <= W[t]
                                                                                    )           


for t in T:                                                                                     
    model += (  lpSum(X[(i,t)]*pWf[i] for i in range(len(M))) <= p*W[t] + oLim*O[t]             
                                                                                    )


In [10]:
# New constraint: Goods park capacity

for t in T:                                                                                     
    model += (  lpSum(X[(i,t)]*sr[i]  for i in range(len(M))) <= gp     + E[t]
                                                                                    )

In [11]:
model

aggregate-planning:
MINIMIZE
1*E_01 + 1*E_02 + 1*E_03 + 1*E_04 + 1*E_05 + 1*E_06 + 1*E_07 + 1*E_08 + 1*E_09 + 1*E_10 + 1*E_11 + 1*E_12 + 1400*F_01 + 1400*F_02 + 1400*F_03 + 1400*F_04 + 1400*F_05 + 1400*F_06 + 1400*F_07 + 1400*F_08 + 1400*F_09 + 1400*F_10 + 1400*F_11 + 1400*F_12 + 800*H_01 + 800*H_02 + 800*H_03 + 800*H_04 + 800*H_05 + 800*H_06 + 800*H_07 + 800*H_08 + 800*H_09 + 800*H_10 + 800*H_11 + 800*H_12 + 250*I_b_01 + 250*I_b_02 + 250*I_b_03 + 250*I_b_04 + 250*I_b_05 + 250*I_b_06 + 250*I_b_07 + 250*I_b_08 + 250*I_b_09 + 250*I_b_10 + 250*I_b_11 + 250*I_b_12 + 500*I_p_01 + 500*I_p_02 + 500*I_p_03 + 500*I_p_04 + 500*I_p_05 + 500*I_p_06 + 500*I_p_07 + 500*I_p_08 + 500*I_p_09 + 500*I_p_10 + 500*I_p_11 + 500*I_p_12 + 140.0*O_01 + 140.0*O_02 + 140.0*O_03 + 140.0*O_04 + 140.0*O_05 + 140.0*O_06 + 140.0*O_07 + 140.0*O_08 + 140.0*O_09 + 140.0*O_10 + 140.0*O_11 + 140.0*O_12 + 420*W_01 + 420*W_02 + 420*W_03 + 420*W_04 + 420*W_05 + 420*W_06 + 420*W_07 + 420*W_08 + 420*W_09 + 420*W_10 + 420*W_11 

In [12]:
model.solve()
LpStatus[model.status]

'Optimal'

In [13]:
print("z* = ", value(model.objective))

z* =  3155116.3799999994


In [14]:
for v in model.variables():
    print(v.name, " = ", v.varValue)

E_01  =  960.0
E_02  =  900.0
E_03  =  780.0
E_04  =  700.0
E_05  =  580.0
E_06  =  891.579
E_07  =  900.0
E_08  =  908.421
E_09  =  960.053
E_10  =  968.474
E_11  =  1312.58
E_12  =  1278.89
F_01  =  11.3889
F_02  =  2.94444
F_03  =  0.0
F_04  =  0.0
F_05  =  0.0
F_06  =  0.0
F_07  =  0.0
F_08  =  0.0
F_09  =  0.0
F_10  =  0.0
F_11  =  0.0
F_12  =  0.0
H_01  =  0.0
H_02  =  0.0
H_03  =  0.0
H_04  =  0.0
H_05  =  0.0
H_06  =  0.0
H_07  =  0.0
H_08  =  0.0
H_09  =  4.05833
H_10  =  0.0
H_11  =  0.0
H_12  =  0.0
I_b_00  =  8.0
I_b_01  =  0.0
I_b_02  =  0.0
I_b_03  =  0.0
I_b_04  =  0.0
I_b_05  =  0.0
I_b_06  =  12.7895
I_b_07  =  10.7895
I_b_08  =  8.0
I_b_09  =  17.5013
I_b_10  =  10.2132
I_b_11  =  0.527632
I_b_12  =  0.0
I_p_00  =  3.0
I_p_01  =  0.0
I_p_02  =  0.0
I_p_03  =  0.0
I_p_04  =  0.0
I_p_05  =  0.0
I_p_06  =  0.0
I_p_07  =  0.0
I_p_08  =  0.0
I_p_09  =  0.0
I_p_10  =  0.0
I_p_11  =  0.0
I_p_12  =  0.0
O_01  =  0.0
O_02  =  0.0
O_03  =  0.0
O_04  =  0.0
O_05  =  0.0
O_06  = 

In [15]:
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack} 
     for name, c in model.constraints.items()]
print(pd.DataFrame(o))


    name shadow price slack
0    _C1         None  None
1    _C2         None  None
2    _C3         None  None
3    _C4         None  None
4    _C5         None  None
..   ...          ...   ...
70  _C71         None  None
71  _C72         None  None
72  _C73         None  None
73  _C74         None  None
74  _C75         None  None

[75 rows x 3 columns]
